In [1]:
import os
import re
import spacy 
from spacy import displacy
import json
import gensim
from gensim.corpora import Dictionary
from gensim.models import LdaModel, CoherenceModel, LsiModel, HdpModel
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
import pandas as pd
from gensim.models import CoherenceModel
import pyLDAvis
import pyLDAvis.gensim_models

c:\Users\Ng_ho\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Load the JSON file into a dictionary
with open('../data/prediction_data/issues.json') as f:
    issues_data = [json.loads(line) for line in f]
issues_df = pd.DataFrame(issues_data)

# Load the JSON file into a dictionary
with open('../data/rawish_data/facts.json') as f:
    facts_data = [json.loads(line) for line in f]
raw_facts_df = pd.DataFrame(facts_data)

# Merge DataFrames
processed_df = pd.merge(raw_facts_df, issues_df, on='casename', how='outer')

# Display the resulting DataFrame
print(processed_df.head())

           casename                                              facts  \
0   2000_SGCA_1.pdf  The facts\nThe appellant is the widow of one T...   
1  2000_SGCA_10.pdf  facts and surrounding circumstances including ...   
2  2000_SGCA_11.pdf  Background \nThe first appellants, a French co...   
3  2000_SGCA_12.pdf  Background\nMicrosoft, Adobe and Autodesk are ...   
4  2000_SGCA_13.pdf  facts. Mere assertion would not suffice. In ex...   

                                              issues  
0  The claim was dismissed with costs by the\nHig...  
1  the claim and\nagainst that decision this appe...  
2  The appeal \nThe questions which arise in this...  
3  the appeals from the assistant registrar. In h...  
4  the appeal on 24 January 2000 and dismissed it...  


In [4]:
def preprocess_text(text):
    text = str(text).lower()
    text = re.sub(r'\W', ' ', text)
    text = re.sub(r'\s+', ' ', text)
    text = re.sub(r'[0-9]', '', text)
    text = re.sub(r'\W*\b(?!no)\w{1,2}\b', '', text)
    stop_words = set(stopwords.words('english'))
    legal_stopwords = ('appellant', 'respondent', 'plaintiff', 'defendant', 'mr', 'mrs', 'dr', 'mdm', 'court','version', 'hr', 'would', 'case', 'sghc', 'court', 'sgca', 'slr', 'sgdc', 'also', 'first', 'person', 'statement', 'line', 'para', 'fact', 'one', 'may', 'time', 'could', 'next', 'legal', 'issues', 'issue')
    stop_words.update(legal_stopwords)
    lemmatizer = WordNetLemmatizer()
    words = text.split()
    words = [lemmatizer.lemmatize(word) for word in words if word not in stop_words]
    
    return words

In [17]:
data = []

processed_df['processed_facts'] = processed_df['facts'].apply(preprocess_text)
processed_df.drop(columns=['facts'], inplace=True)
print(processed_df["processed_facts"])
print("Finished preprocessing text")

texts, article = [], []
print("Performing topic modelling")
for fact in processed_df['processed_facts']:
    for word in fact:
        article.append(word)
        
    texts.append(article)
    article = []

dictionary = Dictionary(texts)
corpus = [dictionary.doc2bow(text) for text in texts]

#initialize parameters
best_coherence = -1
best_lda = None
for num_topics in range(3, 31, 1):
    lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                                id2word=dictionary,
                                                num_topics=num_topics,
                                                random_state=42)
    
    # Get coherence score for each loop
    coherence_model_lda = CoherenceModel(model=lda_model, texts=texts, dictionary=dictionary, coherence='c_v')
    coherence_score = coherence_model_lda.get_coherence()
    
    print(f"Number of topics: {num_topics}, Coherence Score: {coherence_score}")
    if coherence_score > best_coherence:
        best_coherence = coherence_score
        best_topic = num_topics
print(f"Best no of topic: {best_topic} and Best Coherence Score: {best_coherence}")

#use the best model (result from above codes: 10 topics)
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                                id2word=dictionary,
                                                num_topics=best_topic,
                                                random_state=42)

#inspiration from https://www.machinelearningplus.com/nlp/topic-modeling-visualization-how-to-present-results-lda-models/
topics_matrix = lda_model[corpus]
topics = []
# Iterate over each document's topic distribution
# Get the topic with the highest probability
for doc in topics_matrix:
    topic = max(doc, key=lambda x: x[1])[0]
    topics.append(topic)

processed_df['facts_topic'] = topics
processed_df = processed_df.reset_index(drop=True)
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(lda_model, corpus, dictionary)
vis

0       [fact, widow, tan, geok, tee, deceased, sue, c...
1       [fact, surrounding, circumstance, including, a...
2       [background, appellant, french, company, secon...
3       [background, microsoft, adobe, autodesk, compa...
4       [fact, mere, assertion, suffice, exh, said, st...
                              ...                        
8515    [fact, accused, low, sze, song, low, year, old...
8516    [fact, giving, opinion, representation, amount...
8517    [fact, party, karan, bagga, litigant, proceedi...
8518                                                   []
8519    [fact, party, towa, company, incorporated, jap...
Name: processed_facts, Length: 8520, dtype: object
Finished preprocessing text
Performing topic modelling
Number of topics: 3, Coherence Score: 0.39324704976329333
Number of topics: 4, Coherence Score: 0.42437353670658984
Number of topics: 5, Coherence Score: 0.4170010346025063
Number of topics: 6, Coherence Score: 0.39440271026639556
Number of topics: 7, Cohe

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
19    -0.090747 -0.007507       1        1  10.756310
5      0.196475 -0.047952       2        1   9.358062
10     0.026475  0.015073       3        1   9.140808
12    -0.102060 -0.009758       4        1   7.552096
6     -0.072089  0.060286       5        1   7.532726
3     -0.097432 -0.052530       6        1   5.899045
9     -0.053923 -0.070753       7        1   5.262325
0     -0.081567  0.060241       8        1   4.793104
13     0.132158  0.005840       9        1   4.681862
7     -0.060039 -0.093777      10        1   4.628959
16    -0.033856  0.093334      11        1   4.587050
15    -0.060312 -0.014013      12        1   4.135355
17     0.090396  0.077210      13        1   3.575432
11     0.119741 -0.054586      14        1   3.348513
18    -0.082216  0.011344      15        1   3.196358
8      0.124580 -0.051546      16        1   2.969785
1     -0.031891 -0.010549      17        1   2.664422
2      0.104304  0.084759      18        1   2.630301
4     -0.032743 -0.034539      19        1   2.009253
14     0.004746  0.039423      20        1   1.278236, topic_info=          Term          Freq         Total Category  logprob  loglift
812    accused  26434.000000  26434.000000  Default  30.0000  30.0000
3278    victim   9900.000000   9900.000000  Default  29.0000  29.0000
1737  property  17653.000000  17653.000000  Default  28.0000  28.0000
44       share  16538.000000  16538.000000  Default  27.0000  27.0000
3793      drug   8017.000000   8017.000000  Default  26.0000  26.0000
...        ...           ...           ...      ...      ...      ...
701    account    164.531812  16868.345721  Topic20  -5.9519  -0.2704
453        two    163.186714  15162.315218  Topic20  -5.9601  -0.1720
102      claim    155.842661  13742.853771  Topic20  -6.0061  -0.1197
25     defence    147.518582  10426.423371  Topic20  -6.0610   0.1015
239       july    146.208964  10183.327251  Topic20  -6.0699   0.1162

[1625 rows x 6 columns], token_table=       Topic      Freq    Term
term                          
15166     17  0.056052   aaron
15166     20  0.896827   aaron
14784      1  0.005793      ab
14784      3  0.005793      ab
14784      4  0.005793      ab
...      ...       ...     ...
11936     19  0.573484    zhou
24133     14  0.979799  ziploc
24133     20  0.009701  ziploc
20097     17  0.029056     zul
20097     18  0.900747     zul

[12533 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[20, 6, 11, 13, 7, 4, 10, 1, 14, 8, 17, 16, 18, 12, 19, 9, 2, 3, 5, 15])

In [5]:
#Same concept as above
data = []
processed_df['processed_issues'] = processed_df['issues'].apply(preprocess_text)
processed_df.drop(columns=['issues'], inplace=True)
print(processed_df["processed_issues"])
print("Finished preprocessing text")

texts, article = [], []
print("Performing topic modelling")
for issue in processed_df['processed_issues']:
    for word in issue:
        article.append(word)
        
    texts.append(article)
    article = []

dictionary = Dictionary(texts)
corpus = [dictionary.doc2bow(text) for text in texts]

best_coherence = -1
best_lda = None
for num_topics in range(3, 31, 1):
    lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                                id2word=dictionary,
                                                num_topics=num_topics,
                                                random_state=42)
    coherence_model_lda = CoherenceModel(model=lda_model, texts=texts, dictionary=dictionary, coherence='c_v')
    coherence_score = coherence_model_lda.get_coherence()
    
    print(f"Number of topics: {num_topics}, Coherence Score: {coherence_score}")
    if coherence_score > best_coherence:
        best_coherence = coherence_score
        best_topic = num_topics
print(f"Best no of topic: {best_topic} and Best Coherence Score: {best_coherence}")

#use the best model (result from above codes: 25 topics)
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                                id2word=dictionary,
                                                num_topics=best_topic,
                                                random_state=42)

#inspiration from https://www.machinelearningplus.com/nlp/topic-modeling-visualization-how-to-present-results-lda-models/
topics_matrix = lda_model[corpus]
topics = []
# Iterate over each document's topic distribution
# Get the topic with the highest probability
for doc in topics_matrix:
    topic = max(doc, key=lambda x: x[1])[0]
    topics.append(topic)
    
processed_df['issues_topic'] = topics
processed_df = processed_df.reset_index(drop=True)
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(lda_model, corpus, dictionary)
vis

0       [claim, dismissed, cost, high, decision, fook,...
1       [claim, decision, appeal, brought, background,...
2       [appeal, question, arise, appeal, follows, app...
3       [appeal, assistant, registrar, ground, judgmen...
4       [appeal, january, dismissed, give, reason, evi...
                              ...                        
8515    [sub, arise, consideration, whether, low, siva...
8516    [claim, conspiracy, defraud, fault, ken, sally...
8517    [relating, defence, justification, qualified, ...
8518                                                   []
8519    [background, dispute, towa, commenced, suit, a...
Name: processed_issues, Length: 8520, dtype: object
Finished preprocessing text
Performing topic modelling
Number of topics: 3, Coherence Score: 0.34343459736129206
Number of topics: 4, Coherence Score: 0.3511693705252339
Number of topics: 5, Coherence Score: 0.3545368694872686
Number of topics: 6, Coherence Score: 0.3614588493271913
Number of topics: 7, Coher

PreparedData(topic_coordinates=              x         y  topics  cluster      Freq
topic                                               
27     0.041439 -0.101847       1        1  7.306739
0      0.090913 -0.036221       2        1  7.180528
21     0.073708 -0.078849       3        1  6.104266
8      0.078790  0.007044       4        1  4.838791
10     0.050430  0.032918       5        1  4.828442
9      0.008654  0.073656       6        1  4.670875
25    -0.104689 -0.111581       7        1  4.622186
7      0.018807  0.018524       8        1  4.396503
23     0.041557  0.072459       9        1  4.240962
1      0.062723  0.005121      10        1  4.118423
17     0.026485  0.072258      11        1  4.070495
24    -0.130869 -0.028481      12        1  3.678165
2      0.029010  0.022453      13        1  3.663860
16    -0.024780 -0.089126      14        1  3.623359
11    -0.077000 -0.052986      15        1  3.597802
26     0.005114  0.063817      16        1  3.386443
5      0.057712 -0.013638      17        1  3.083519
15    -0.128971  0.013364      18        1  2.793990
14     0.060829 -0.012062      19        1  2.762845
3      0.004294  0.037240      20        1  2.398572
13    -0.053009  0.057628      21        1  2.187880
18    -0.009220 -0.018861      22        1  2.024265
12     0.051473  0.027854      23        1  1.919137
22    -0.007786  0.016169      24        1  1.819865
28    -0.188418  0.043488      25        1  1.815112
4     -0.034989 -0.019631      26        1  1.737299
19     0.042291 -0.079651      27        1  1.548249
20     0.042969 -0.001350      28        1  1.079617
6     -0.027466  0.080290      29        1  0.501811, topic_info=          Term          Freq          Total Category  logprob  loglift
1151   accused  53408.000000   53408.000000  Default  30.0000  30.0000
210    company  81361.000000   81361.000000  Default  29.0000  29.0000
1147   account  77893.000000   77893.000000  Default  28.0000  28.0000
2263   offence  34260.000000   34260.000000  Default  27.0000  27.0000
801   property  54246.000000   54246.000000  Default  26.0000  26.0000
...        ...           ...            ...      ...      ...      ...
1074       two    377.565943   58319.051594  Topic29  -5.7801   0.2548
44     alleged    324.118444   29183.787735  Topic29  -5.9328   0.7944
191      claim    381.934856  118407.200125  Topic29  -5.7686  -0.4419
505    however    350.443135   57413.524591  Topic29  -5.8547   0.1959
748      party    346.074022  132344.548712  Topic29  -5.8672  -0.6518

[2476 rows x 6 columns], token_table=       Topic      Freq    Term
term                          
24559     12  0.055764   abang
24559     15  0.008579   abang
24559     18  0.926534   abang
24559     21  0.008579   abang
34610     20  0.886371  abject
...      ...       ...     ...
1142      23  0.007511   zoned
1142      28  0.007511   zoned
25704      2  0.008131  zoning
25704     10  0.967604  zoning
25704     22  0.008131  zoning

[27261 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[28, 1, 22, 9, 11, 10, 26, 8, 24, 2, 18, 25, 3, 17, 12, 27, 6, 16, 15, 4, 14, 19, 13, 23, 29, 5, 20, 21, 7])

In [7]:
processed_df.drop(columns=['processed_facts'], inplace=True)
processed_df.drop(columns=['processed_issues'], inplace=True)
processed_df.to_csv("issues_facts_topics.csv")